In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
import time
import pandas as pd
from numpy import e

In [2]:
def job_posts(page_number=2,profession='Data Scientist',location='United States'):       # Define a function that takes page number, profession and location as arguments.
    if ' ' in profession:                                                                # Optimize the link for LinkedIn. 
        profession = profession.replace(' ','%20')                     
    if ' ' in location:                                                                  # Optimize the link for LinkedIn. 
        location = location.replace(' ','%20')
    driver = webdriver.Chrome()                                                          # Instantiate the Chrome browser
    links = []                                                                           # Create an empty links list to use later as storage.
    position_names = []                                                                  # Create an empty position names list to use later as storage.
    companies = []                                                                       # Create an empty companies list to use later as storage.
    locations = []                                                                       # Create an empty locations list to use later as storage.
    path1 = f'https://www.linkedin.com/jobs/search?keywords={profession}&location={location}'  # The standard link sequence which can be modified by the user inputs 
    driver.get(path1)                                                                    # Browser navigates to the link
    i = 0                                                                                # Base number that helps with arranging the iteration number
    while i < page_number:                                                               # While loop that helps with scrolling. 
        html = driver.find_element(By.TAG_NAME,'html')                                   # Select anything on the page
        html.send_keys(Keys.END)                                                         # Send 'END' key to scroll down the page 
        i = i+1                                                                          # i increases incrementally. This helps with the number of repeats
        try:                                                                             # If the HTML does not include the desired element, the flow is protected with try func.
            t = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/button')# Find 'See more Jobs' button
            t.click()                                                                    # Click the button
            time.sleep(2)                                                                # Let the page load for 2 seconds.
        except:                                                                          # If a problem arises the flow continues. 
            pass
            time.sleep(3)                                                                # Wait for 3 secs
    block = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/ul')        # Find 
    job_list = block.find_elements(By.CLASS_NAME, 'base-card')
    try:
        for job in job_list:
            try:
                link = job.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href')
                links.append(link)
            except:
                links.append(None)
                ValueError('no link')
            try:
                position_name = job.find_element(By.TAG_NAME,'span').text
                position_names.append(position_name)
            except:
                position_names.append(None)
                ValueError('no pos name')
            try:
                company = job.find_element(By.CLASS_NAME,'hidden-nested-link').text
                companies.append(company)
            except:
                companies.append(None)
                ValueError('no company')
            try:
                location = job.find_element(By.CLASS_NAME,'job-search-card__location').text
                locations.append(location)
            except:
                locations.append(None)
                ValueError('no location')
    except:
        e = 'no path'
        ValueError(e)
        print(e)
    data = pd.DataFrame({'Position':position_names,
            'Company':companies,
            'Location':locations,
            'Link':links})     
    return data

In [3]:
df=job_posts(2,profession='Analyst',location='England')

In [4]:
def get_content(P_no=2,prof='Data Scientist',loc='United States'):
    df = job_posts(page_number = P_no,profession=prof,location=loc)
    links=df.Link.tolist()
    driver = webdriver.Chrome()
    levels = []
    linkss =[] 
    employment = []
    job_functions = []
    industries = []
    contents = []
    for link in links:
        if len(contents)%25 == 0 and len(contents)>0:
            print(f'Scraping job number:{len(contents)}...')    
        try:
            driver.get(link)
            driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
            time.sleep(1)     
            try:
                linkss.append(link)
            except:
                linkss.append(None)
                ValueError('No link')
            try:
                Seniority_Level = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[1]/span').text
                levels.append(Seniority_Level)
            except:
                levels.append(None)
                ValueError('No level')
            try:
                Employment_Type = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[2]/span').text
                employment.append(Employment_Type)
            except:
                employment.append(None)
                ValueError('no employment')  
            try:
                Job_Function = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[3]/span').text
                job_functions.append(Job_Function)
            except:
                job_functions.append(None)
                ValueError('no job function')
            try:
                Industry = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[4]/span').text
                industries.append(Industry)
            except:
                industries.append(None)
                ValueError('no industry')     
            try:
                content=driver.find_element(By.CLASS_NAME,'show-more-less-html__markup').text
                contents.append(content)
            except:
                contents.append(None)
                ValueError('no content')     
        except:
            ValueError('smthng is wrong')
        time.sleep(5)
    c_data = pd.DataFrame({'links':linkss,
                            'content':contents,
                            'level':levels,
                            'job_function':job_functions,
                            'industries':industries,
                            'employment':employment})
    df_merged = c_data.merge(df,how='left',right_on='Link',left_on='links')
    df_merged = df_merged.drop(columns=['Link','links'])
    return df_merged

In [5]:
get_content(P_no=2,prof='Data Scientist',loc='United States')

Scraping job number:25...
Scraping job number:50...


,content,level,job_function,industries,employment,Position,Company,Location
0,About The Position\n\nThe Golden State Warrior...,Entry level,Engineering and Information Technology,Spectator Sports,Full-time,Data Scientist,Golden State Warriors,"San Francisco, CA"
1,Description\n\nBroadlume’s on a mission to tra...,Entry level,Engineering and Information Technology,"Technology, Information and Internet",Full-time,Data Scientist (Remote),Broadlume,"New York, NY"
2,Job Summary\n\nThe Data Science Institute (DSI...,Entry level,Engineering and Information Technology,Higher Education,Part-time,DATA SCIENTIST,University of Wisconsin-Madison,"Madison, WI"
3,NO C2C!!!\n\nAlderson Loop is looking for a Da...,Mid-Senior level,"Analyst, Information Technology, and Health Ca...",IT Services and IT Consulting and Hospitals an...,Contract,Data Scientist,Alderson Loop,United States
4,Desired Skills and Experience *\nInsight Globa...,Mid-Senior level,Information Technology,IT Services and IT Consulting,Full-time,Data Scientist,Insight Global,United States
...,...,...,...,...,...,...,...,...
70,Job Description\n\nDiverse Lynx LLC is an Equa...,Entry level,Engineering and Information Technology,Software Development,Full-time,Data Scientist,Diverse Lynx,"Austin, TX"
71,Job Description\n\nThe Data scientist will ass...,Entry level,Engineering and Information Technology,Software Development,Contract,Data Scientist,Diverse Lynx,"Fort Worth, TX"
72,Job Description\n\nTechnical Skillset (Mandato...,Entry level,Engineering and Information Technology,Software Development,Contract,Data Scientist,Diverse Lynx,"Franklin, TN"
73,Job Description\n\nDATA SCIENTIST\n\nANYWHERE ...,Entry level,Engineering and Information Technology,Software Development,Contract,Data Scientist,Diverse Lynx,"Kansas City, MO"
